 * https://www.kaggle.com/ivanegapratama/covid-eda-initial-exploration-tool
 * https://www.kaggle.com/jonathanbesomi/cord-19-sources-unification-with-pyspark-sql

In [46]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import glob
import json

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [48]:
spark = SparkSession.builder.appName("HW3-data-exploration").getOrCreate()

In [49]:
root_path = '../data/archive/'
metadata_path = f'{root_path}/metadata.csv'
print(metadata_path)

../data/archive//metadata.csv


In [50]:
meta_df = spark.read.option("header",True).csv(metadata_path)
meta_df.limit(10).toPandas()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,None,None,None,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,None
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,None,None,None,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,None,None,None,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,None,None,None,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,None,None,None,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
5,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,None,None,None,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,None
6,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,None,None,None,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,None
7,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,None,None,None,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,None
8,8qnrcgnk,faaf1022ccfe93b032c5608097a53543ba24aedb,PMC,Heme oxygenase-1 and carbon monoxide in pulmon...,10.1186/1465-9921-4-7,PMC193681,12964953,no-cc,"Heme oxygenase-1 (HO-1), an inducible stress p...",2003-08-07,"Slebos, Dirk-Jan; Ryter, Stefan W; Choi, Augus...",Respir Res,None,None,None,document_parses/pdf_json/faaf1022ccfe93b032c56...,document_parses/pmc_json/PMC193681.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1..

In [51]:
print("There are ", meta_df.count(), " sources in the metadata file.")

There are  315948  sources in the metadata file.


In [52]:
meta_df.printSchema()

root
 |-- cord_uid: string (nullable = true)
 |-- sha: string (nullable = true)
 |-- source_x: string (nullable = true)
 |-- title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubmed_id: string (nullable = true)
 |-- license: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- mag_id: string (nullable = true)
 |-- who_covidence_id: string (nullable = true)
 |-- arxiv_id: string (nullable = true)
 |-- pdf_json_files: string (nullable = true)
 |-- pmc_json_files: string (nullable = true)
 |-- url: string (nullable = true)
 |-- s2_id: string (nullable = true)



Get all of the JSON files

In [53]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
print("There are ", len(all_json), "sources files.")

There are  209980 sources files.


In [54]:
#todo - for now restrict this to 100 files
all_json = all_json[:100]

In [55]:
data = spark.read.json(all_json, multiLine=True)
data.createOrReplaceTempView("data")

In [56]:
data.limit(10).toPandas()

,abstract,back_matter,bib_entries,body_text,metadata,paper_id,ref_entries
0,"[([], [], Abstract, In eukaryotic cells, prote...",[],"(([Row(first='A', last='Kanapin', middle=[], s...","[([], [], Introduction, Cells are internally c...","([(Row(institution='University of Cambridge', ...",00154ef181732195c77fc1abd791fa2a996b9099,"((None, Fusion of post-Golgi carriers in mamma..."
1,"[([], [], Abstract, Dendritic cells (DCs) are ...",[],"(([Row(first='R', last='Steinman', middle=['M'...","[([Row(end=1139, ref_id='BIBREF0', start=1136,...","([(Row(institution=None, laboratory=None, loca...",00142f93c18b07350be89e96372d240372437ed9,"((None, Properties and functions of human and ..."
2,"[([], [], Abstract, The bloodstream is the mai...",[],"(([Row(first='H', last='Wolinsky', middle=[], ...","[([Row(end=355, ref_id=None, start=350, text='...",([(Row(institution='University of Pennsylvania...,002c6a7e4b04193230529c0313c0368b2de6936a,"((None, Inflammation affects surface expressio..."
3,"[([], [], Abstract, Scutellaria baicalensis Ge...","[([], [], Acknowledgements, Not applicable. ),...","(([], , (None, None), , b0, Commission CP. Pha...","[([Row(end=260, ref_id='BIBREF0', start=257, t...","([(Row(DOI=None, PMID=None, arXiv=None), , Jia...",002f32c1ed13a1f56a64eb286dd243d4c7a1e4fd,"((None, Chemical structure of components of SB..."
4,"[([], [], Abstract, Microbial life permeates E...","[([], [], Acknowledgments, This article is an ...","(([Row(first='M', last='Ader', middle=[], suff...","[([], [], , specific scale; (2) identifying th...","([(Row(institution='University of Utah', labor...",002c8c43cea5f3ec8c97d30fe181c01b919d789a,"((None, Iron oxide mineral precipitates have v..."
5,"[([], [], Abstract, Lack of safe and effective...",[],"(([Row(first='A', last='Mitchell', middle=['M'...","[([Row(end=162, ref_id='BIBREF0', start=159, t...","([(Row(institution='Albany Medical College', l...",0024b4d3d69e815b4d8579ee253acd83061445fa,"((None, Trivalent-FP induced enhanced protecti..."
6,"[([], [], Abstract, SARS-CoV-2 infection is ch...",[],"(([Row(first='W', last='Guan', middle=[], suff...","[([Row(end=200, ref_id='BIBREF0', start=197, t...","([(Row(DOI=None, PMID=None, arXiv=None), , Car...",0017447a237cee0445a83538a538e7c6e9336de6,"(None, None, None, None, None, None, None, Non..."
7,"[([], [], Abstract, Dengue has a negative impa...",[],"(([Row(first='S', last='Bhatt', middle=[], suf...","[([Row(end=525, ref_id=None, start=521, text='...",([(Row(institution='The Australian National Un...,0022796bb2112abd2e6423ba2d57751db06049fb,"((None, Number of locally acquired and importe..."
8,"[([], [], Abstract, Viruses are small infectio...","[([], [], acknowledgement, Acknowledgements Th...","(([Row(first='T', last='Abbink', middle=['E'],...","[([Row(end=896, ref_id='BIBREF97', start=878, ...","([(Row(DOI=None, PMID=None, arXiv=None), , Eli...",001b7a3388e3c1e712a01e30cdd8dbd3e8535e1e,"((None, Examples of non-canonical viral mRNA t..."
9,"[([], [], Abstract, Reçu et accepté le 7 févri...",[],"(([Row(first='K', last='Hamprecht', middle=[],...","[([], [], Introduction, Les pathologies infect...",([(Row(institution='centre hospitalier de Tour...,000eec3f1e93c3792454ac59415c928ce3a6b4ad,"((None, novembre 2002. Il apparaît dans le sud..."


In [64]:
# Select text columns
covid_sql = spark.sql(
        """
        SELECT
            metadata.title AS title,
            abstract.text AS abstract, 
            body_text.text AS body_text,
            back_matter.text AS back_matter,
            paper_id
        FROM data
        """)

In [65]:
covid_sql.limit(10).toPandas()

,title,abstract,body_text,back_matter,paper_id
0,Journal Pre-proof Direct trafficking pathways ...,"[In eukaryotic cells, protein sorting is a hig...",[Cells are internally compartmentalised. This ...,[],00154ef181732195c77fc1abd791fa2a996b9099
1,immunity to pathogens taught by specialized hu...,[Dendritic cells (DCs) are specialized antigen...,[iNTRODUCTiON Human beings are constantly expo...,[],00142f93c18b07350be89e96372d240372437ed9
2,Journal Pre-proof Targeting drug delivery in t...,[The bloodstream is the main transporting path...,"[epitopes, for many reasons including rather r...",[],002c6a7e4b04193230529c0313c0368b2de6936a
3,"Applications, phytochemistry, pharmacological ...",[Scutellaria baicalensis Georgi. (SB) is a com...,[written by many medical scientists together d...,"[Not applicable. , Not applicable., Not applic...",002f32c1ed13a1f56a64eb286dd243d4c7a1e4fd
4,Deciphering Biosignatures in Planetary Contexts,[Microbial life permeates Earth's critical zon...,[specific scale; (2) identifying the precise c...,[This article is an outgrowth of an internatio...,002c8c43cea5f3ec8c97d30fe181c01b919d789a
5,Preclinical Efficacy of a Trivalent Human FcγR...,[Lack of safe and effective mucosal adjuvants ...,[Mucosal surfaces are prone to infection by ma...,[],0024b4d3d69e815b4d8579ee253acd83061445fa
6,COVID-19 as part of the hyperferritinemic synd...,[SARS-CoV-2 infection is characterized by a pr...,[The outbreak of the SARS-CoV-2 virus emerged ...,[],0017447a237cee0445a83538a538e7c6e9336de6
7,Public Health Responses to and Challenges for ...,[Dengue has a negative impact in low-and lower...,[Pathogens and vectors can now be transported ...,[],0022796bb2112abd2e6423ba2d57751db06049fb
8,Hidden Silent Codes in Viral Genomes,[Viruses are small infectious agents that repl...,[Proteins are the principal actors in all intr...,[Acknowledgements This study was supported in ...,001b7a3388e3c1e712a01e30cdd8dbd3e8535e1e
9,Pneumonie virale sévère de l'immunocompétent V...,"[Reçu et accepté le 7 février 2004, Les infect...",[Les pathologies infectieuses respiratoires so...,[],000eec3f1e93c3792454ac59415c928ce3a6b4ad


In [66]:
covid_sql.printSchema()

root
 |-- title: string (nullable = true)
 |-- abstract: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- body_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- back_matter: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- paper_id: string (nullable = true)



Now that we have the following:
  * title,
  * abstract,
  * body text,
  * back_matter

Join all of them

We definitely need to keep:
  * paper_id (as an index)
  * abstract
  * body text
  

Adding the Word Count Column

In [67]:
word_join_f = F.udf(lambda x: [''.join(w) for w in x], StringType())

In [68]:
covid_sql = covid_sql.withColumn("abstract", word_join_f("abstract"))
covid_sql = covid_sql.withColumn("body_text", word_join_f("body_text"))

In [69]:
# see https://stackoverflow.com/questions/48927271/count-number-of-words-in-a-spark-dataframe
df_with_wc = covid_sql.withColumn('wordCount_abstract', F.size(F.split(F.col('abstract'), ' ')))
df_with_wc = df_with_wc.withColumn('wordCount_body_text', F.size(F.split(F.col('body_text'), ' ')))

df_with_wc.limit(10).toPandas()

,title,abstract,body_text,back_matter,paper_id,wordCount_abstract,wordCount_body_text
0,Journal Pre-proof Direct trafficking pathways ...,"[In eukaryotic cells, protein sorting is a hig...",[Cells are internally compartmentalised. This ...,[],00154ef181732195c77fc1abd791fa2a996b9099,316,8738
1,immunity to pathogens taught by specialized hu...,[Dendritic cells (DCs) are specialized antigen...,[iNTRODUCTiON Human beings are constantly expo...,[],00142f93c18b07350be89e96372d240372437ed9,309,5306
2,Journal Pre-proof Targeting drug delivery in t...,[The bloodstream is the main transporting path...,"[epitopes, for many reasons including rather r...",[],002c6a7e4b04193230529c0313c0368b2de6936a,189,8444
3,"Applications, phytochemistry, pharmacological ...",[Scutellaria baicalensis Georgi. (SB) is a com...,[written by many medical scientists together d...,"[Not applicable. , Not applicable., Not applic...",002f32c1ed13a1f56a64eb286dd243d4c7a1e4fd,271,5480
4,Deciphering Biosignatures in Planetary Contexts,[Microbial life permeates Earth's critical zon...,[specific scale; (2) identifying the precise c...,[This article is an outgrowth of an internatio...,002c8c43cea5f3ec8c97d30fe181c01b919d789a,220,14073
5,Preclinical Efficacy of a Trivalent Human FcγR...,[Lack of safe and effective mucosal adjuvants ...,[Mucosal surfaces are prone to infection by ma...,[],0024b4d3d69e815b4d8579ee253acd83061445fa,722,7132
6,COVID-19 as part of the hyperferritinemic synd...,[SARS-CoV-2 infection is characterized by a pr...,[The outbreak of the SARS-CoV-2 virus emerged ...,[],0017447a237cee0445a83538a538e7c6e9336de6,171,4709
7,Public Health Responses to and Challenges for ...,[Dengue has a negative impact in low-and lower...,[Pathogens and vectors can now be transported ...,[],0022796bb2112abd2e6423ba2d57751db06049fb,276,7290
8,Hidden Silent Codes in Viral Genomes,[Viruses are small infectious agents that repl...,[Proteins are the principal actors in all intr...,[Acknowledgements This study was supported in ...,001b7a3388e3c1e712a01e30cdd8dbd3e8535e1e,205,5912
9,Pneumonie virale sévère de l'immunocompétent V...,"[Reçu et accepté le 7 février 2004, Les infect...",[Les pathologies infectieuses respiratoires so...,[],000eec3f1e93c3792454ac59415c928ce3a6b4ad,413,6150
